<a href="https://colab.research.google.com/github/JHMamun/Performance_testing_NB_SVM/blob/main/lstm_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
import numpy as np
import pandas as pd

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,WordNetLemmatizer
from nltk.tokenize import word_tokenize
import sklearn.metrics as m

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving emails.csv to emails.csv


In [ ]:
dataset = pd.read_csv('emails.csv', encoding = 'latin-1')
dataset

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1
...,...,...
5723,Subject: re : research and development charges...,0
5724,"Subject: re : receipts from visit jim , than...",0
5725,Subject: re : enron case study update wow ! a...,0
5726,"Subject: re : interest david , please , call...",0


In [ ]:
sent = dataset.iloc[:,[0]]['text']
sent

0       Subject: naturally irresistible your corporate...
1       Subject: the stock trading gunslinger  fanny i...
2       Subject: unbelievable new homes made easy  im ...
3       Subject: 4 color printing special  request add...
4       Subject: do not have money , get software cds ...
                              ...                        
5723    Subject: re : research and development charges...
5724    Subject: re : receipts from visit  jim ,  than...
5725    Subject: re : enron case study update  wow ! a...
5726    Subject: re : interest  david ,  please , call...
5727    Subject: news : aurora 5 . 2 update  aurora ve...
Name: text, Length: 5728, dtype: object

In [ ]:
labels = dataset.iloc[:,[1]]['spam']
labels

0       1
1       1
2       1
3       1
4       1
       ..
5723    0
5724    0
5725    0
5726    0
5727    0
Name: spam, Length: 5728, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
labels = le.fit_transform(labels)

In [ ]:
le.classes_

array([0, 1])

In [ ]:
labels

0       1
1       1
2       1
3       1
4       1
       ..
5723    0
5724    0
5725    0
5726    0
5727    0
Name: spam, Length: 5728, dtype: int64

In [ ]:
import re

In [ ]:
lemma = WordNetLemmatizer()

In [ ]:
sentences=[]
for j in range(0,len(sent)):
  s=re.sub('[^A-Za-z]',' ',sent[j])
  s=s.lower()
  words=word_tokenize(s)
  words=[lemma.lemmatize(i) for i in words if i not in stopwords.words('english')]
  s=' '.join(words)
  sentences.append(s)

In [ ]:
from keras.preprocessing.text import one_hot

In [ ]:
one_hot_sent = [one_hot(i, 10000) for i in sentences]

In [ ]:
maxi = [len(i) for i in one_hot_sent]

In [ ]:
max(maxi)

4109

In [28]:
from keras.preprocessing.sequence import pad_sequences

In [29]:
padsequences = pad_sequences(one_hot_sent,maxlen=80)

In [30]:
padsequences

array([[1189, 9753, 9626, ..., 3843, 7142,  860],
       [   0,    0,    0, ...,  260, 3158, 6125],
       [   0,    0,    0, ..., 2182, 9221,  676],
       ...,
       [1833, 4659, 5996, ..., 3219, 1215, 9279],
       [9710, 2277, 5880, ..., 4759, 8282, 9710],
       [ 809, 2150, 5458, ...,  809, 6128, 4785]], dtype=int32)

In [31]:
feature_train,feature_test,label_train,label_test=train_test_split(padsequences,labels,test_size=0.2,random_state=7)

In [32]:
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,Dropout,Flatten

In [33]:
model=Sequential()
model.add(Embedding(10000,64,input_length=80))
model.add(LSTM(100))
model.add(Dense(2,activation='sigmoid'))
model.compile(loss='binary_crossentropy',metrics=['accuracy'],optimizer='adam')

In [36]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 80, 64)            640000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               66000     
_________________________________________________________________
dense (Dense)                (None, 2)                 202       
Total params: 706,202
Trainable params: 706,202
Non-trainable params: 0
_________________________________________________________________


In [35]:
model.fit(feature_train,label_train,epochs=20,validation_data=(feature_test,label_test))

Epoch 1/20


ValueError: ignored